# Сбор данных и фрагментация
**Оставлю из урока так как есть, что бы помнить**

Для сбора и подготовки данных нам необходимо:
* Собрать все заголовки статей заданной категории статей Википедии, заходя на первый уровень вложенности статей.
* Разделить каждую статью на разделы (секции) и подразделы. Отбросить менее релевантные разделы, такие как внешние ссылки и сноски.
* Очистить текст, удалив теги ссылок, пробелы и очень короткие секции (разделы).
* Добавить заголовки и подзаголовки к тексту каждого раздела, чтобы помочь chatGPT понять контекст.

### Сбор заголовков статей из категории

In [ ]:
# Отключим предупреждения в колабе. Будет меньше лишней информации в выводе
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# installing libraries
!pip install openai mwclient mwparserfromhell tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 11.4 MB/s eta 0:00:00


In [ ]:
# imports libraries
import mwclient  # библиотека для работы с MediaWiki API для загрузки примеров статей Википедии
import mwparserfromhell  # Парсер для MediaWiki
import openai  # будем использовать для токинизации
import pandas as pd  # В DataFrame будем хранить базу знаний и результат токинизации базы знаний
import re  # для вырезания ссылок <ref> из статей Википедии
import tiktoken  # для подсчета токенов
import time  # Импортируем модуль time для задеркжи парсера

# Делаю под русско-язычную Википедию
*Добавлю проверку на существующую страницу и обработку исключений и ошибок try-except*

In [ ]:
# parsing
# Задаём категорию и сайт русской Википедии
CATEGORY_TITLE = "Медицина"  # Указываем здесь без слова Категория - Category
WIKI_SITE = "ru.wikipedia.org"

# Подключаемся к русской версии Википедии
site = mwclient.Site(WIKI_SITE)

# Убедимся, что страница действительно существует на русском языке.
category_page = site.categories[CATEGORY_TITLE]
if not category_page.exists:
    raise ValueError(f"Категория '{CATEGORY_TITLE}' не найдена.")

# Функция сбора заголовков всех статей в указанной категории и её подкатегориях
def titles_from_category(
    category: mwclient.listing.Category,  # Задаем типизированный параметр категории статей
    max_depth: int  # Определяем глубину вложения статей
) -> set[str]:
    """
    Возвращает множество заголовков страниц в данной категории Википедии и её подкатегориях.

    :param category: Объект категории из библиотеки mwclient
    :param max_depth: Максимальное количество уровней вложенности категорий
    :return: Множество заголовков статей
    """
    titles = set()  # Используем множество для хранения заголовков статей
    for item in category.members():
        if isinstance(item, mwclient.page.Page):
            try:
                # Попытка получить текст страницы
                text = item.text()
                # Добавляем название страницы в множество заголовков
                titles.add(item.name)
                # Добавляем задержку в 1 секунду между запросами
                time.sleep(1)
            except Exception as e:
                print(f"Произошла ошибка при получении текста страницы '{item.name}': {e}")
        elif isinstance(item, mwclient.listing.Category) and max_depth > 0:
            # Рекурсивно обрабатываем подкатегорию
            deeper_titles = titles_from_category(item, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles

# Собираем заголовки статей из данной категории и одного уровня вложенных подкатегорий
titles = titles_from_category(category_page, max_depth=1)

# Печать результата
print(f"Собрано {len(titles)} заголовков статей в категории '{CATEGORY_TITLE}'.")

Собрано 155 заголовков статей в категории 'Медицина'.


### Извлечение секций из документов

Определим секции в документах, которые менее релевантны и их можно отбросить:
Так как использую русско-язычную версию Википедии, нужно и отображать секции на русском языке



In [ ]:
# section_ejection
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]

Код зависит от правильной обработки русского текста. Я использую библиотеку mwparserfromhell, которая отлично справляется с разбором структуры страниц Википедии независимо от языка.

In [ ]:
# ettings_functions
# Функция возвращает список всех вложенных секций для заданной секции страницы Википедии
def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,  # Объект секции, который нужно обработать
    parent_titles: list[str],  # Список родительских заголовков
    sections_to_ignore: set[str],  # Набор заголовков секций, которые нужно игнорировать
) -> list[tuple[list[str], str]]:
    """
    Из раздела Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """
    # Извлекаем заголовки из секции
    headings = [str(h) for h in section.filter_headings()]
    # Получаем заголовок текущей секции
    title = headings[0].strip("=" + " ")

    # Если заголовок в списке игнорируемых, возвращаем пустой список
    if title in sections_to_ignore:
        return []

    # Обновляем список заголовков, добавляя текущий
    titles = parent_titles + [title]
    # Получаем полный текст секции
    full_text = str(section)
    # Извлекаем текст текущей секции, исключая заголовок
    section_text = full_text.split(title)[1]

    # Если в секции только один заголовок, возвращаем его текст
    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        # Иначе, находим первый подзаголовок
        first_subtitle = headings[1]
        # Извлекаем текст до первого подзаголовка
        section_text = section_text.split(first_subtitle)[0]
        # Добавляем текущую секцию в результаты
        results = [(titles, section_text)]
        # Рекурсивно обрабатываем все вложенные секции
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results


# Функция возвращает список всех секций страницы, за исключением тех, которые отбрасываем
def all_subsections_from_title(
    title: str,  # Заголовок страницы Википедии
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,  # Набор заголовков секций, которые нужно игнорировать
    site_name: str = WIKI_SITE,  # Имя сайта Википедии
) -> list[tuple[list[str], str]]:
    """
    Из заголовка страницы Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """
    # Подключаемся к сайту Википедии
    site = mwclient.Site(site_name)
    # Получаем страницу по заголовку
    page = site.pages[title]

    # Если страница не существует, выбрасываем исключение
    if not page.exists:
        raise ValueError(f"Страница '{title}' не найдена.")

    # Получаем текст страницы
    text = page.text()
    # Парсим текст страницы
    parsed_text = mwparserfromhell.parse(text)
    # Извлекаем заголовки из текста
    headings = [str(h) for h in parsed_text.filter_headings()]

    # Если есть заголовки, извлекаем текст до первого заголовка
    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        # Если заголовков нет, весь текст является аннотацией
        summary_text = str(parsed_text)

    # Добавляем аннотацию страницы в результаты
    results = [([title], summary_text)]
    # Обрабатываем все секции уровня 2
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results

In [ ]:
#into_sections
# Разбивка статей на секции
# придется немного подождать, так как на парсинг 100 статей требуется около минуты
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
    time.sleep(3)  # Добавляем задержку в 3 секунды после обработки каждой статьи
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")

Найдено 978 секций на 155 страницах


### Очистка текста (теги ссылок, пробелы и короткие секции)

In [ ]:
# learing_text
# Очистка текста секции от ссылок <ref>xyz</ref>, начальных и конечных пробелов
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section
    # Удаляем ссылки
    text = re.sub(r"<ref.*?</ref>", "", text)
    # Удаляем пробелы вначале и конце
    text = text.strip()
    return (titles, text)

# Применим функцию очистки ко всем секциям с помощью генератора списков
wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

# Отфильтруем короткие и пустые секции
def keep_section(section: tuple[list[str], str]) -> bool:
    """Возвращает значение True, если раздел должен быть сохранен, в противном случае значение False."""
    titles, text = section
    # Фильтруем по произвольной длине, можно выбрать и другое значение
    if len(text) < 16:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Отфильтровано {original_num_sections-len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")

Отфильтровано 91 секций, осталось 887 секций.


Для наглядности выведем 5 секций, заголовок и текст (100 первых символов) каждой секции

In [ ]:
for ws in wikipedia_sections[10:30]:
    print(ws[0])
    display(ws[1][:100] + "...")
    print()

['Послеоперационные активаторы', 'Ссылки']


'==\n* [https://www.nose-fit.com www.nose-fit.com]\n* https://www.sciencedirect.com/science/article/abs...'


['Категория:Медицинское страхование']


'{{Родственные проекты}}\n{{Основная статья}}\n\n[[Категория:Социальное страхование]]\n[[Категория:Медици...'


['Демаркационная линия (медицина)']


...

'==\nЗа последние 20 лет дети получили значительные улучшения как в технологии, так и в клиническом ве...'


['Почечный диализ', 'Диализ в разных странах', 'В Соединённом Королевстве']


'===\nНациональная служба здравоохранения обеспечивает диализ в Соединённом Королевстве. В Англии услу...'


['Почечный диализ', 'Диализ в разных странах', 'В Соединённых Штатах']


'===\nС 1972\xa0года Соединённые Штаты покрывают стоимость диализа и трансплантации для всех граждан. К 2...'

### Фрагментация документов

Поскольку chatGPT может одновременно считывать только ограниченный объем токенов, нам необходимо разделить каждый документ на фрагменты (части).

Далее мы рекурсивно разделим длинные разделы на более мелкие. Идеального рецепта разделения текста на разделы не существует.

Однако, можно сделать следующие предположения:

* Более длинные разделы могут быть лучше для вопросов, требующих большего контекста.
* Более длинные разделы могут быть хуже для поиска, поскольку в них может быть больше запутанных тем.
* Более короткие разделы лучше подходят для снижения затрат, которые пропорциональны количеству токенов.
* Более короткие разделы позволяют извлекать больше разделов для запроса, что может помочь с полнотой данных.
* Перекрывающиеся разделы могут помочь предотвратить обрезание ответов по смыслу границами разделов.

Учитывая все выше сказанное, мы будем использовать простой подход и ограничим количество разделов до 1600 токенов в каждом, рекурсивно сокращая вдвое любые слишком длинные разделы. Во избежание сокращения в середине полезных предложений, мы будем разделять секции на части по границам абзацев.

In [ ]:
# Document_fragmentation
GPT_MODEL = "gpt-3.5-turbo"  # имеет значение только в той мере, в какой он выбирает, какой токенизатор использовать
# Явно указываем токенизатор
tokenizer = tiktoken.get_encoding("cl100k_base")
# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str, # строка
    model: str, # модель
    max_tokens: int, # максимальное число разрешенных токенов
    print_warning: bool = True, # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string

# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: tuple[list[str], str], # секции
    max_tokens: int = 1000, # максимальное число токенов
    model: str = GPT_MODEL, # модель
    max_recursion: int = 5, # максимальное число рекурсий
) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

In [ ]:
# Делим секции на части
MAX_TOKENS = 2000
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")

887 секций Википедии поделены на 912 строк.


In [ ]:
# Напечатаем пример строки
print(wikipedia_strings[2])

Женская больница Мулаго

Обзор

==
Строительство больницы началось в апреле 2013 года, а ввод в эксплуатацию изначально ожидался во второй половине 2016 года. После задержек строительство было завершено в июле 2018 года.

Женская больница Мулаго — это [[Реферативная практика (медицина)|реферативное]] медицинское учреждение, женское отделение Национальной больницы Мулаго. В его состав входят дородовые клиники, родильные отделения, операционные, палаты послеродового наблюдения и послеродовые отделения. Больница обслуживает пациентов акушерско-гинекологического профиля. В больнице имеется онкологическое отделение, предназначенное для пациентов с гинекологическими онкологическими заболеваниями, включая рак яичников, рак фаллопиевых труб, рак матки, рак эндометрия, рак шейки матки, рак влагалища и злокачественные новообразования вульвы. 

В больнице будут оборудованные по последнему слову техники отделения для новорожденных, отделения для недоношенных детей, отделение интенсивной терапии дл

### Токенизация и сохранение результата

Теперь, когда мы разделили нашу базу знаний на более короткие автономные строки, мы можем вычислять эмбединги для каждой строки.

In [ ]:
# Tokenization_save
from openai import OpenAI
import os
import pandas as pd
import getpass

# Модель эмбеддингов
EMBEDDING_MODEL = "text-embedding-ada-002"

# Получаем ключ API от пользователя
os.environ["VSEGPT_API_KEY"] = getpass.getpass("Введите VseGPT API Key:")

# Создаем клиент OpenAI с настройкой на использование API VseGPT
client = OpenAI(
    api_key=os.environ.get("VSEGPT_API_KEY"),
    base_url="https://api.vsegpt.ru/v1",
)

# Функция для вычисления эмбеддингов строки
def get_embedding(text, model=EMBEDDING_MODEL):
    return client.embeddings.create(input=[text], model=model).data[0].embedding

Введите VseGPT API Key:··········


In [ ]:
df = pd.DataFrame({"text": wikipedia_strings[:15]})

df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

SAVE_PATH = "./medicine_knowledge_2022.csv"
# Сохранение результата
df.to_csv(SAVE_PATH, index=False)

Сохраню базу знаний в CSV-файле.

Для больших наборов данных необходимо использовать векторную базу данных, которая будет более производительной.

In [ ]:
df.head(10)

,text,embedding
0,Женская больница Мулаго\n\n{{Универсальная кар...,"[-0.02458954229950905, 0.012538427487015724, -..."
1,Женская больница Мулаго\n\nРасположение\n\n==\...,"[-0.005425150506198406, 0.017862308770418167, ..."
2,Женская больница Мулаго\n\nОбзор\n\n==\nСтроит...,"[-0.01604926772415638, 0.011421728879213333, -..."
3,Женская больница Мулаго\n\nРаспределение коек\...,"[-0.013167863711714745, 0.009832487441599369, ..."
4,Женская больница Мулаго\n\nРуководство\n\n==\n...,"[-0.026457738131284714, -0.0018055977998301387..."
5,Женская больница Мулаго\n\nСм. также\n\n==\n\n...,"[-0.01516191940754652, 0.00015063137107063085,..."
6,Женская больница Мулаго\n\nПримечания\n\n==\n<...,"[-0.013714994303882122, 0.007285228464752436, ..."
7,Женская больница Мулаго\n\nСсылки\n\n==\n\n* [...,"[-0.016681814566254616, 0.006670073606073856, ..."
8,Послеоперационные активаторы\n\n<noinclude>{{к...,"[-0.0186057910323143, 0.0111607126891613, -0.0..."
9,Послеоперационные активаторы\n\nПримечания\n\n...,"[-0.022903330624103546, 0.019568033516407013, ..."


Подгружаю данные из CSV-файла и преобразуем строковые представления списков в настоящие списки, в столбце embedding.

In [ ]:
import ast
embeddings_path = "./medicine_knowledge_2022.csv"

df = pd.read_csv(embeddings_path)

# Конвертируем наши эмбединги из строк в списки
df['embedding'] = df['embedding'].apply(ast.literal_eval)

# **Метод Search**
Далее, определим функцию поиска, которая:

* Принимает пользовательский запрос и DataFrame со столбцами `text` и `embedding`;
* Токенизирует пользовательский запрос с помощью OpenAI API (для этого используется токенизатор `text-embedding-ada-002`);
* Использует косинусное расстояние для определения схожести между токенизированым пользовательским запросом и эмбендингами в DataFrame для ранжирования текстов по схожести или релевантности;
* Возвращает два списка:
  * N лучших текстов, ранжированных по релевантности;
  * Их соответствующие оценки релевантности.

In [ ]:
from scipy import spatial  # вычисляет сходство векторов
EMBEDDING_MODEL = "text-embedding-ada-002"

# Функция поиска
def strings_ranked_by_relatedness(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), # функция схожести, косинусное расстояние
    top_n: int = 100 # выбор лучших n-результатов
) -> tuple[list[str], list[float]]: # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""

    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding

    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

# **Метод Ask**

С помощью функции поиска, описанной выше, мы теперь можем автоматически извлекать необходимые сведения из DataFrame и вставлять их в сообщения для chatGPT.

Ниже мы определяем функцию ask, которая:

* Принимает запрос пользователя
* Ищет в базе знаний текст релевантный запросу
* Вставляет этот текст в сообщение для GPT
* Отправляет сообщение в GPT
* Возвращает ответ GPT

In [ ]:
# устанавливаем библиотеку
!pip install tiktoken

In [ ]:
# Ask_method
# с этой функцией мы уже знакомы
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке для заданной модели"""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    model: str, # модель
    token_budget: int # ограничение на число отсылаемых токенов в модель
) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df) # функция ранжирования базы знаний по пользовательскому запросу
    # Шаблон инструкции для chatGPT
    message = 'Используйте приведенные ниже статьи о Медицине, чтобы ответить на следующий вопрос. Если ответ не найден в статьях, напишите "Я не смог найти ответ".'
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str, # пользовательский запрос
    df: pd.DataFrame = df, # DataFrame со столбцами text и embedding (база знаний)
    model: str = GPT_MODEL, # модель
    token_budget: int = 4096 - 1000, # ограничение на число отсылаемых токенов в модель
    print_message: bool = False, # нужно ли выводить сообщение перед отправкой
) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, model=model, token_budget=token_budget)
    # Если параметр True, то выводим сообщение
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Ты большой знаток медицины, и если нет информации в базе данных, ты ответишь сам."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create (
    model = model,
    messages = messages,
    temperature = 0 # гиперпараметр степени случайности при генерации текста. Влияет на то, как модель выбирает следующее слово в последовательности.
    )
    response_message = response.choices[0].message.content
    return response_message

# Создаю Телеграмм бота

Уклон бота будет поиск информации по медецине

In [ ]:
# Установка библиотек
!pip install aiogram -q
!pip install aiosqlite -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.2/698.2 kB 13.0 MB/s eta 0:00:00


In [ ]:
# Установим библиотеку nest_asyncio
!pip install nest_asyncio -q
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Импорт библиотек
import asyncio
import logging
import aiosqlite
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram import types
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram import F
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton

In [ ]:
# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)
# Ваш токен бота
API_TOKEN = '7566236365:AAGHsZRfG_QThq_pMAnbp39_S6GzqzkKN9M'

# Создаем объекты бота и диспетчера
bot = Bot(token=API_TOKEN)
dp = Dispatcher()

# Хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
       builder = ReplyKeyboardBuilder()
       builder.add(types.KeyboardButton(text="Задать вопрос"))
       await message.answer(
           "Привет! Я бот, знающий информацию о медицине. В будущем я смогу давать рекомендации по твоему здоровью, а пока ты можешь у меня спросить всё, что касается медицины. "
           "Чтобы получить информацию по взаимодействию с ботом, введите /help.",
           reply_markup=builder.as_markup(resize_keyboard=True)
       )

@dp.message(Command('help'))
async def help_me(message: types.Message):
       await message.answer(f'Тематика бота: медицина')
       await message.answer(f'Число записей в базе знаний: {df.shape[0]}')
       await message.answer(f'Пример запроса: Расскажи о направлениях и областях в медицине')

@dp.message()
async def handle_message(message: types.Message):
       if message.text == "Задать вопрос":
           await ask_question_prompt(message)
       else:
           await ask_question(message)

async def ask_question_prompt(message: types.Message):
       await message.answer("Задавайте вопрос, я посмотрю, есть ли в моей базе знаний ответы.")

async def ask_question(message: types.Message):
       user_query = message.text
       try:
           await message.answer("Один момент, поиск информации...")
           answer = ask(user_query)
           await message.answer(answer)
       except Exception as e:
           await message.answer(f"Произошла ошибка: {str(e)}")

# Основной цикл программы
async def main():
       await dp.start_polling(bot)

if __name__ == '__main__':
       asyncio.run(main())